# Setup


In [1]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking

# We enable tip and volume tracking globally using the `set_volume_tracking` and `set_tip_tracking` methods.
set_volume_tracking(enabled=True)
set_tip_tracking(enabled=True)

In [2]:
from pylabrobot.liquid_handling.backends.piper_backend import PiperBackend
import logging

# 10.1.0.4
# 192.168.11.221

config = {
  "dry": True, "verbose": True,
  "ws_address": "", "sio_address": "",
  # "ws_address": "ws://localhost:7125/websocket",
  # "sio_address": "http://localhost:3333",
  # "loglevel": 3,
  # "verbose": True
  "datatools": "mongo",
  "database": {
    "database_url": 'localhost:27017',
    # "env_file": "~/pipettin-bot/code/pipettin-gui/.env",
    "database_name": 'pipettin'
  }
}

back = PiperBackend(config=config)

In [ ]:
from pylabrobot.resources.pipettin import SilverDeck

# Instantiate the SilverDeck.
deck = SilverDeck(config=config, workspace_name="Experiment")
deck.summary()

In [ ]:
from pylabrobot.liquid_handling import LiquidHandler

lh = LiquidHandler(backend=back, deck=deck)

await lh.setup()

In [ ]:
await back._home_machine()

# Send commands

In [ ]:
tip_rack = deck.get_resource("Blue tip rack")

tip_rack.print_grid()

In [ ]:
back.channels

In [ ]:
pickups = await lh.pick_up_tips(tip_rack["A3"], use_channels=back.channels.PMULTI)

In [2]:
# TODO: Chequear que piper esté usando el tip que le pasé.

In [ ]:
tube_rack = deck["Tube Rack [5x16]"]

aspirations = await lh.aspirate(tube_rack["A1"], vols=[100], use_channels=back.channels.PMULTI)

In [ ]:
well_plate = deck["Standard 384-well plate"]

for well in well_plate["B2:B7"]:
  dispenses = await lh.dispense([well], vols=[15], use_channels=back.channels.PMULTI)

In [ ]:
trash = deck["Tall Trash"]

await lh.drop_tips([trash], use_channels=back.channels.PMULTI, allow_nonzero_volume=True)

# Close

In [ ]:
await lh.stop()